In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os

downloads = 'C:/Users/balob/Downloads'

#### To get the path

In [3]:
from pathlib import Path
path_dir = Path.cwd()
print(path_dir)
path_dir

C:\Users\balob\Documents\GITLAB\RB_BD


WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

In [2]:
pwd=!echo %cd%
pwd=str(pwd)
pwd=r"c:\Users\balob\Documents\GITLAB\NOC_Roamability_com"
pwd.replace('\\','/',100)

'c:/Users/balob/Documents/GITLAB/NOC_Roamability_com'

In [ ]:
%reset -f

# Examples

### Fetch data from MS SQL DB

In [ ]:
#Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT mcc.mcc,c.country_name FROM RDB_NETWORK_IMSI_PREFIXES mcc
LEFT JOIN RDB_NETWORKS net
ON mcc.NETWORK_ID=net.NETWORK_ID
LEFT JOIN RDB_COUNTRIES c
ON c.country_id=net.country_id
'''

df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
#df_tadig['TADIG_CODE_ID']=df_tadig['TADIG_CODE_ID'].astype('int')
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
cnxn.close()

### Fetch data from Maria DB

In [ ]:
import mysql.connector #pip install mysql-connector-python-rf
import pandas as pd

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig
,CostTariffId
,MIN((Cost*1024*1024)/TotalChargedUnits) AS Price_min
,AVG((Cost*1024*1024)/TotalChargedUnits) AS Price_avg
,STDDEV_SAMP((Cost*1024*1024)/TotalChargedUnits) AS Price_std
,MAX((Cost*1024*1024)/TotalChargedUnits) AS Price_max
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20180810000000
AND VisitedNetworkTadig = 'BHSBH'
GROUP BY VisitedNetworkTadig,CostTariffId
'''

df_mariadb = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()
df_mariadb.head()

### Soap API OCS Portal

In [184]:
#%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = 'prod' #'prod' or 'lab'

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

You are connected to Reseller: WMB Limited RT.    
Available credit for the Reseller: 291 USD.

The Reseller contains the following accounts:
 0    WM&B Limited Test SIM (accountID: 352496)
 1    WMB 5K Simcards order (accountID: 352537)
 2    Hoang Tran (accountID: 352541)
 3    Test1 (accountID: 352542)
 4    Test2 (accountID: 352543)


In [155]:
subscriber_imsi = client.service.getSubscriberByIMSI('425019613026594')
subscriber_imsi

{
    'result': {
        'code': '6',
        'description': 'Internal error',
        'errorUUID': 'cb1fa827-c1b7-4456-8388-888b6174f946'
    },
    'subscriber': None
}

### Testing DMI Steering API

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
url = 'http://roamdb.roamability.com/SteeringAPI/Test'
payload = {'SponsorIMSI':'425019613990511',
           'RealIMSI':'425019613990511',
           'Mcc':'262',
           'SessionID':'1234567'}
response = requests.post(url,payload)
soup = BeautifulSoup(response.text, 'xml')
net_list = []
for net in soup.find_all('Network'):
    net_list.append([net.Rank.text,net.Mcc.text,net.Mnc.text,net.Name.text,net.Support4G.text,net.Discounted.text,net.Cost.text])
DataFrame(net_list,columns=['Rank','MCC','MNC','Operator_name','4G_Support','Discounted','Cost'])

### Send MSU

In [179]:
from bs4 import BeautifulSoup
import roamability as rb
reload(rb)

ogt = {'p4':'48790993070',
       'partner':'97254120624',
       'sure':'447797706411',
       'tot':'66893773228'}

rb.test_func()
rb.print_imsi_prof('48790993070', '260060149000585', '79112009610', 'as_resp')

#soup = BeautifulSoup(rb.sri4sm(ogt, msisdn),'xml')
#soup = BeautifulSoup(rb.prn(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sai(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sendSMS(ogt, imsi, msc, payload),'xml')
#soup

Greetings from Roamability!
<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response> 

No info in response.


 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [80]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
any_tags  = ['#sim_card_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\SIM_Applet\_TEMPLATE.txt
----------------------------------------------------------------------------------------------------
#sim_card_principle_of_operation
#sim_card_mop
------------------------------
Аплеты, загружаемые на карты.

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\SIM_Reader\_TEMPLATE.txt
----------------------------------------------------------------------------------------------------
#activation_principle_of_operation
#sim_card_principle_of_operation
#sim_card_mop
------------------------------
"Any PCSC reader can be used. I have good experience with ACS (https://www.acs.com.hk/) and OMNIKEY (https://www.hidglobal.com/products/readers/omnikey) readers."
(https://roamability.teamwork.com/#/tasks/12118095)



### Search in ALARMS by folder names

In [75]:
import os

for path in paths:
    names = os.listdir(path)
    [print(path + name) for name in names if name.upper().find('TWNLD')!=-1]

c:/W_DATA_ROAM/ALARMS/PS_Partner_TWNLD_4G_issue_#2399570_190117


### Read csv example

In [ ]:
#Read CSV
df_tadig = pd.read_csv('TADIG.csv',sep=';',header=None,dtype={'MCC':object,'MNC':object},
                   names=['TADIG','MCC','MNC'],skiprows=0)

### Groupby examples

In [ ]:
#Вариант №1
#Группировка с применением различных типов агрегации к колонкам groupby().agg
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']
df.groupby(['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 1 задания группировки)
df.groupby(df['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 2 задания группировки)
df.groupby(df['key1']).agg({'data1': [np.min,np.mean,np.max,np.size], 'data2': [np.min,np.mean,np.max,np.size]}) #Агрегация для каждого поля несколько функций
df.groupby(df['key1']).agg({'data1': [('f_min',np.min),('f_max',np.max)],'data2': [('f_min',np.min),('f_max',np.max)]}) #Агрегация для каждого поля несколько функций с переименованием
df.groupby(['key1','key2']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким колонкам
df.groupby(level=0).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 1 указания индекса)
df.groupby(level='ind1').agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 2 указания индекса)
df.groupby(level=[0,1]).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=[0,1]).agg(['min','max']) #Агрегация для каждого поля несколько функций
df[['data1','data2']].groupby(level=[0,1]).agg([('fmin','min'),('fmax','max')]) #Агрегация для каждого поля несколько функций с переименованием
()

In [ ]:
#Вариант №3
#Группировка .groupby + агрегирующая функция
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']

# The Best Practice
df.groupby(['key1'],as_index=False)['data1'].max()
df.groupby(level=0)['data1'].max().reset_index()

df['data1'].groupby(df['key1']).mean() #Агрегация одного поля по одному ключу
df.groupby(df['key1']).mean() #Агрегация всех полей по одному ключу
df[['data1','data2']].groupby(df['key1']).mean() #Агрегация нескольких полей по одному ключу
df[['data1','data2']].groupby([df['key1'],df['key2']]).mean() #Агрегация нескольких полей по нескольким ключам
df[['data1','data2']].groupby(level=0).mean() #Агрегация по одному индексу (вариант 1 указания индекса)
df[['data1','data2']].groupby(level='ind1').mean() #Агрегация по одному индексу (вариант 2 указания индекса)
df[['data1','data2']].groupby(level=[0,1]).mean() #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=0,axis=1).mean() #Агрегация с группировкой по столбцам
#или
grouped=df['data1'].groupby(df['key1'])
grouped.mean()
#или
df.groupby(df['key1'])['data1'].mean()
#Чтобы добавить префикс к названию строки/столбца
#df['data1'].groupby(df['key1']).mean().add_prefix('mean_')
#df[['data1','data2']].groupby(level=0,axis=1).mean()
df['data1_mean']=df.groupby('key1')['data1'].transform(np.max)
()

### Search in Allowed Lists and others

In [186]:
downloads = 'C:/Users/balob/Downloads'

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Mon_Jan_21_2019.csv'))
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads,'DMI.dmi_netpfx_export_Sat_Dec_29_2018.csv'))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [226]:
#df_al[(df_al['PLMN code'].str.startswith('MKDCC', na=False)) & (df_al['Sponsor']=='')]
df_al.loc[(df_al['PLMN code'].str.startswith('GBRVF', na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','PLMN code']].sort_values(by=['Sponsor','PLMN code'])

,Sponsor,PLMN code
1318,P4_KeepGo_Test,GBRVF
2170,PA_Telzar_Soft,GBRVF
140,Partner,GBRVF
2684,Partner_All,GBRVF
2473,Partner_Dialoq_A,GBRVF
3106,Partner_RedT_Sof,GBRVF
1440,Partner_RedTea,GBRVF
17,Partner_WeConnec,GBRVF
1990,Partner_WeStream,GBRVF
510,Partner_mondic,GBRVF


In [232]:
df_al_new=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Mon_Jan_21_2019_new.csv'))
df_al_new.loc[(df_al_new['PLMN code'].str.startswith('', na=False)) &
          (df_al_new['Sponsor'].str.contains('STI', na=False)), ['Sponsor','PLMN code']].sort_values(by=['Sponsor','PLMN code']).\
to_csv(join(downloads,'result.csv'), index=False)

In [225]:
str(df_np[df_np.Operator == 'TURTK'].Prefix.unique().tolist())
df_np[(df_np.Operator.str.startswith('GBRVF',na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
1658,GBRVF,44385,1,7,234,15
1659,GBRVF,44385,1,149,234,15
1660,GBRVF,447785,1,7,234,15
1661,GBRVF,447785,1,149,234,15
3430,GBRVF,447785019999,0,7,234,15
4115,GBRVF,4477850100,0,7,234,15
4116,GBRVF,447785019999,0,149,234,15
4117,GBRVF,4477850132,0,7,234,15
8268,GBRVF,4477850120,0,7,234,15
8269,GBRVF,4477850121,0,7,234,15


In [215]:
str(df_np[(df_np.Prefix.str.startswith('6683',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('51955',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
12787,PERN3,5195539,1,7,716,17
12789,PERN3,51955392201,0,7,716,17
12790,PERN3,51955392202,0,7,716,17
12794,PERN3,51955392203,0,149,716,17
12795,PERN3,51955392203,0,7,716,17
12796,PERN3,51955392204,0,7,716,17
12797,PERN3,51955392204,0,149,716,17
12798,PERN3,51955399136,0,149,716,17
12799,PERN3,51955399136,0,7,716,17
12800,PERN3,51955399165,0,7,716,17


In [ ]:
df_dmi_diam_oper[(df_dmi_diam_oper['DMI Operator'].str.startswith('', na=False))\
                 & (df_dmi_diam_oper['Realm'].str.contains('405', na=False))]

### Hex to Dec conversion

In [192]:
# Hex to dec
print(int('11CE',16))

# Dec to hex
print(hex(4558))
print('%x' % 272252642)

# Dec to bin
print(bin(3))

# Bin to dec
print(int('01000111101110',2))

4558
0x11ce
103a3ee2
0b11
4590


In [188]:
f'{bin(2)} - {bin(61)} - {bin(6)}'
#2.61.6

'0b10 - 0b111101 - 0b110'

In [186]:
print(bin(10915))
a = '10101010100011'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
5 - 84 - 3


### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [217]:
import paramiko
print_lines = 50
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            if 'Total' in line:
                print(i,line,end='')
            #print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

0 2018-12-14 05:00:45,537 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:44.009712
8 2018-12-13 05:00:45,731 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.732656
16 2018-12-12 05:00:44,040 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.703231
24 2018-12-11 05:00:41,949 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:39.759998
32 2018-12-10 05:00:42,655 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:40.982727
40 2018-12-09 05:00:46,048 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.357029
48 2018-12-08 05:00:44,158 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.308123


### Pricing counting

In [192]:
moc=1.5
mtc=0
text=0.25
data=7.37

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

Price P4
Price 1.2            : [1.8, 0.0, 0.3, 8.844]
Price 1.2*1.5        : [2.7, 0.0, 0.45, 13.266]
Price 1.2*1.5*1.15   : [3.105, 0.0, 0.5175, 15.2559]
Price 1.2*1.5*1.30   : [3.51, 0.0, 0.585, 17.2458]
Price 2              : [3.6, 0.0, 0.6, 17.688]

Price Partner
Price 1.5            : [2.25, 0.0, 0.375, 11.055]
Price 1.5*1.15       : [2.5875, 0.0, 0.43125, 12.71325]
Price 1.5*1.30       : [2.925, 0.0, 0.4875, 14.3715]
Price 2              : [3.0, 0, 0.5, 14.74]


## Testing

In [ ]:
import mglearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [ ]:
import requests
from bs4 import BeautifulSoup
resp = requests.get('http://www.cbr.ru/scripts/XML_daily.asp')
soup = BeautifulSoup(resp.content,'xml')
eur = soup.find(ID='R01239').Value.string
print('Курс EUR на составляет: {} RUB'.format(eur))

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
with sr.Microphone() as source:
    print('Say:')
    audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        print(f'You said: {text}')
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
    except:
        print('Sorry, I cant recognize what you said.')

In [ ]:
from gtts import gTTS 
import os
from os.path import join
downloads = 'C:/Users/balob/Downloads'
mytext = 'Привет'
language = 'en'
language = 'ru'
voice = gTTS(text=mytext, lang=language, slow=False) 
voice.save(join(downloads,'welcome.mp3')) 
os.system(join(downloads,'welcome.mp3'))

In [156]:
tuple_vec = (1, 2, 3)
list_vec = (4, 5, 6)

def print_vector(x, y, z):
    print(f'Vector: <{x}, {y}, {z}>')
print_vector(*tuple_vec)
print_vector(*list_vec)

Vector: <1, 2, 3>
Vector: <4, 5, 6>


In [169]:
dict_vec = {'z':3, 'y':2, 'x':1}
def print_vector(x, y, z):
    print(f'Vector: <{x}, {y}, {z}>')
print_vector(**dict_vec)
print_vector(*dict_vec)

Vector: <1, 2, 3>
Vector: <z, y, x>


In [185]:
def foo1(value):
    """Явное указание 'return None'"""
    if value:
        return value
    else:
        return None
def foo2(value):
    """Пустая инструкция return подразуменвает 'return None'"""
    if value:
        return value
    else:
        return
def foo3(value):
    """Пропущенная инструкция return подразуменвает 'return None'"""
    if value:
        return value
functions = [foo1, foo2, foo3]
for function in functions:
    print(f'Функция {function.__name__}(). {function.__doc__}. Возвращает тип: {type(function(0))}')

Функция foo1(). Явное указание 'return None'. Возвращает тип: <class 'NoneType'>
Функция foo2(). Пустая инструкция return подразуменвает 'return None'. Возвращает тип: <class 'NoneType'>
Функция foo3(). Пропущенная инструкция return подразуменвает 'return None'. Возвращает тип: <class 'NoneType'>


In [23]:
lst = [1, 2, 3, 4, 5]
lst_link = lst
lst_shallow_copy = lst[:]
lst_hard_copy = lst.copy()
lst[:] = [7, 8, 9]
print(lst)
print(lst_link)
print(lst_link)
print(lst_shallow_copy)
print(lst_hard_copy)

[7, 8, 9]
[7, 8, 9]
[7, 8, 9]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]


In [3]:
lst = [1, 2, 3, 4, 5]
b = lst[:]
b

[1, 2, 3, 4, 5]

In [4]:
lst[0] = 99
b

[1, 2, 3, 4, 5]

In [5]:
lst

[99, 2, 3, 4, 5]

In [6]:
b[0] = 100

In [8]:
b

[100, 2, 3, 4, 5]

In [126]:
%reset -f